In [1]:
!pip install tldextract
!pip install transformers
!pip install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.8 MB/s eta 0:00:00


In [2]:
import csv
import ssl
import os
import torch
import ast
from urllib.request import Request, urlopen, URLError, urljoin
from urllib.parse import urlparse
import time
import threading
import queue
from bs4 import BeautifulSoup
from bs4.element import Comment
from pathlib import Path
import re
from datetime import datetime
from queue import PriorityQueue
from matplotlib import pyplot as plt
import tldextract
import hashlib
import requests
from google.colab import files as FILE
import pathlib
import pandas as pd
from fake_useragent import UserAgent
from PIL import Image

from transformers import pipeline
from transformers import CLIPProcessor, CLIPModel
from torch.nn import functional as F


clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [47]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [48]:
### Script Settings ###
conf = dict(
    dir = '/content/drive/My Drive/KAIST/Spider1/',
    imgsDir = "/content/drive/My Drive/KAIST/Spider1/imgs/",
    keywords = ['Elon Musk', 'Tour Eiffel'],
    labels = ['technology','business','politics', 'history'],
    blacklist = ['youtube.com', 'facebook.com',
                   'twitter.com', 'x.com', 'instagram.com', 'linkedin.com','tiktok.com'],
    mode = "summarize",
    image_weight = 0.3
)

###
start_time = int(time.time())
crawled_csv = conf['dir']+'crawler_'+"{}.csv".format(start_time)

In [50]:
#Delete the Crawler Directory to start from a clean dirctory
if os.path.exists(conf['dir']) is False:
  print("Create Directory")
  os.mkdir(conf['dir'])

if os.path.exists(conf['imgsDir']) is False:
  print("Create Imgs Directory")
  os.mkdir(conf['imgsDir'])

crawler_csv_file = open(crawled_csv, mode='a')
crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
crawlerFile_writer.writerow(["timestamp","hash","link","keyword","post_text","downloaded_imgs","processed","label","true_label"])
crawler_csv_file.close()

Create Directory
Create Imgs Directory


In [51]:
from sklearn.metrics.pairwise import cosine_similarity

def get_image_relevance(image, keyword_embed, threshold=0.3):
    """
    Computes the relevance of an image to a keyword.
    Returns a tuple of (similarity, embedding, should_keep_image).
    """

    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_embed = clip_model.get_image_features(**inputs).squeeze(0)

    sim = cosine_similarity(
        [image_embed.cpu().numpy()],
        [keyword_embed.cpu().numpy()]
        )[0][0]



    should_keep_image = sim >= threshold
    return sim, image_embed, should_keep_image

def deduplicate_embeddings(image_embeddings, threshold=0.9):
    if not image_embeddings:
        return []
    vectors = torch.stack(image_embeddings).cpu().numpy()
    sim_matrix = cosine_similarity(vectors)
    keep_indices = []
    discarded = set()

    for i in range(len(vectors)):
        if i in discarded:
            continue
        keep_indices.append(i)
        for j in range(i + 1, len(vectors)):
            if sim_matrix[i][j] >= threshold:
                discarded.add(j)
    return keep_indices


def filter_images(images, keyword, image_threshold=0.3):
    """
    Filters images
    """

    input = clip_processor(text=[keyword], return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        keyword_embed = clip_model.get_text_features(**input).squeeze()

    relevant_images = []
    for image in images:
        result = get_image_relevance(image, keyword_embed, image_threshold)
        if result is None:
            continue
        sim, emb, should_keep_image = result
        if should_keep_image:
            relevant_images.append((image, emb))
            print(f"Image {image} is relevant with similarity {sim:.4f}")
        else:
            print(f"Image {image} is not relevant with similarity {sim:.4f}")

    if not relevant_images:
        return []

    image_embeddings = [emb for _, emb in relevant_images]
    keep_idxs = deduplicate_embeddings(image_embeddings)
    kept_embeds = [image_embeddings[i] for i in keep_idxs]
    kept_images = [relevant_images[i][0] for i in keep_idxs]

    return kept_images


In [ ]:
def chunk_text(text, max_len=77, stride=60):
  """Split long text into overlapping chunks based on tokenizer."""
  encoded = clip_processor(text, return_attention_mask=False, return_token_type_ids=False, add_special_tokens=False)
  input_ids = encoded['input_ids']

  chunks = []
  for i in range(0, len(input_ids), stride):
    chunk_ids = input_ids[i:i + max_len]
    if not chunk_ids:
      continue
    chunk_text = clip_processor.decode(chunk_ids, clean_up_tokenization_spaces=True)
    chunks.append(chunk_text)
    if len(chunk_ids) < max_len:
      break
  return chunks

def summarize_text(text, max_tokens=70):
    """Perform light summarization on the input text."""

    tokenizer = summarizer.tokenizer
    inputs = tokenizer.encode(text, truncation=True, max_length=1000)
    truncated_text = tokenizer.decode(inputs, skip_special_tokens=True)

    summary = summarizer(
        truncated_text,
        max_length=max_tokens,
        min_length=20,
        do_sample=False
    )[0]['summary_text']

    return summary

def get_label(sentence, images=None, image_weight=0.5, mode = None):
  max_len = 77

  if mode == "summarize":
      summarized = summarize_text(sentence, max_tokens=60)
      texts = [summarized] + conf['labels']

      text_inputs = clip_processor(
          text=texts,
          return_tensors="pt",
          padding=True,
          truncation=True,
          max_length=max_len
      )

      with torch.no_grad():
        text_outputs = clip_model.get_text_features(**text_inputs)

      sentence_emb = text_outputs[0:1]
      label_embs = text_outputs[1:]

  elif mode == "chunks":
    sentence_chunks = chunk_text(sentence)
    texts = sentence_chunks + conf['labels']

    text_inputs = clip_processor(
        text=texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
    )

    with torch.no_grad():
      text_outputs = clip_model.get_text_features(**text_inputs)

    chunk_embs = text_outputs[:len(sentence_chunks)]
    label_embs = text_outputs[len(sentence_chunks):]
    sentence_emb = chunk_embs.mean(dim=0, keepdim=True)

  else:
      texts = [sentence] + conf['labels']
      text_inputs = clip_processor(
        text=texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
      )

  if images:
    image_inputs = clip_processor(
        images=images,
        return_tensors="pt"
    )
    with torch.no_grad():
        image_outputs = clip_model.get_image_features(**image_inputs)
    image_emb = image_outputs.mean(dim=0, keepdim=True)

    combined_emb = (1 - image_weight) * sentence_emb + image_weight * image_emb

  else:
    combined_emb = sentence_emb

  combined_emb = F.normalize(combined_emb, dim=-1)
  label_embs = F.normalize(label_embs, dim=-1)

  similarities = F.cosine_similarity(combined_emb, label_embs)
  closest = similarities.argsort(descending=True)

  top_indices = closest[:2]

  top_labels = {
      conf['labels'][i]: similarities[i].item()
      for i in top_indices
  }

  return top_labels


import json
from collections import defaultdict

def run(save_path):
  df = pd.read_csv(crawled_csv,index_col=False)
  grouped_links = defaultdict(list)
  for i in range(len(df. index)):
    sentence = df.loc[i, 'post_text']
    keyword = df.loc[i, 'keyword']
    images_names = df.loc[i, 'downloaded_imgs']
    images_names = ast.literal_eval(images_names)
    images_paths = [conf['imgsDir'] + name for name in images_names]

    valid_images = []
    for path in images_paths:
        try:
            img = Image.open(path).convert("RGB")
            if img.mode == 'P':
                img = img.convert('RGBA')
            else:
                img = img.convert('RGB')
            valid_images.append(img)
        except:
            print(f"Skipped file {path}")

    filtered_images = filter_images(valid_images, keyword, image_threshold=0.3)

    prediction = get_label(sentence, filtered_images, conf["image_weight"], conf["mode"])
    top_label = max(prediction, key=prediction.get)
    link = df.loc[i, 'link']
    grouped_links[top_label].append(link)
  with open(save_path, 'w', encoding='utf-8') as f:
      json.dump(grouped_links, f, ensure_ascii=False, indent=4)
      print(f"✅ Grouped links saved to {conf['dir']}output.json")


In [53]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    if re.match(r"[\n]+", str(element)):
        return False
    return True

def scrape_and_save(link, keyword, conf, crawled_csv):
    my_ssl = ssl.create_default_context()
    my_ssl.check_hostname = False
    my_ssl.verify_mode = ssl.CERT_NONE

    timestamp = datetime.timestamp(datetime.now())

    try:
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        response = urlopen(req, context=my_ssl)
        soup = BeautifulSoup(response.read(), "html.parser")

        hash_val = int(hashlib.sha256(link.encode('utf-8')).hexdigest(), 16) % 10**8

        # ----- Image scraping -----
        images = soup.find_all('img')
        downloaded_imgs = []

        for i, image in enumerate(images):
            image_link = None
            for attr in ["data-srcset", "data-src", "data-fallback-src", "src"]:
                if image.get(attr):
                    image_link = image[attr]
                    break

            if image_link:
                try:
                    content = requests.get(image_link).content
                    ext = pathlib.Path(image_link).suffix
                    if ext in [".png", ".jpg"]:
                        file_name = f"{conf['imgsDir']}{hash_val}_{i}{ext}"
                        with open(file_name, "wb") as f:
                            f.write(content)
                        downloaded_imgs.append(f"{hash_val}_{i}{ext}")
                except Exception as e:
                    pass

        # ----- Text scraping -----
        texts = soup.find_all(string=True)
        visible_texts = filter(tag_visible, texts)
        cleaned_texts = [t.strip() for t in visible_texts if len(t.strip().split()) > 5]
        post_text = ' '.join(cleaned_texts)

        if len(post_text.split()) < 6:
            print('Text too short, skipping:', link)
            return

        # ----- Write to CSV -----
        with open(crawled_csv, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([timestamp, hash_val, link, keyword, post_text, downloaded_imgs, "No", "N/A", " "])

        print(f"Done scraping {link}")

    except URLError as e:
        print(f"URL {link} error: {e.reason}")
    except Exception as e:
        print(f"Unexpected error on {link}: {e}")

In [54]:
blocked_domains = conf['blacklist']


def duckduckgo_search(query, max_results=10, retries=100):
    tries = 0
    results = []
    seen_links = set()

    ua = UserAgent()
    session = requests.Session()

    while tries <= retries and len(results) < max_results:
        try:
            url = "https://duckduckgo.com/html/"
            params = {'q': query, 'kl': 'us-en'}
            headers = {'User-Agent': ua.random}
            response = session.get(url, headers=headers,
                                   params=params, timeout=5)

            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all(
                'a', attrs={'class': 'result__a'}, href=True)
            if not links:
                raise Exception(
                    "No links found — possibly blocked or unexpected page structure.")

            if len(links) < max_results:
                print(
                    f"Less links than max_results, setting max_results to {len(links)}.")
                max_results = len(links)

            for link in links:
                url = link['href']
                title = link.get_text(strip=True)

                parsed_url = urlparse(url)
                if not parsed_url.scheme or not parsed_url.netloc:
                    continue

                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"

                if not any(parsed_url.netloc.endswith(domain) for domain in blocked_domains):
                    if url not in seen_links:
                        seen_links.add(url)
                        results.append({
                            'title': title,
                            'base_url': base_url,
                            'link': url,
                            'keyword': query
                        })
                        if len(results) >= max_results:
                            break
            tries += 1
            time.sleep(0.6)

        except Exception as e:
            print("Failed to scrape the page:", e)
            print("Retries left:", retries-tries)
            tries += 1
            time.sleep(0.6)

    if len(results) == 0:
        raise Exception(f"Max retries exceeded: {retries}")

    return results


In [ ]:
print("The Crawler has the following configuraions:")
print(conf)


keywords = conf["keywords"]

all_results = []

for keyword in keywords:
    print(f"Searching for keyword: {keyword}")
    try:
        results = duckduckgo_search(keyword)
        all_results.extend(results)  # add results for this keyword
    except Exception as e:
        print(f"Search failed for keyword '{keyword}': {e}")

url_list = [result['link'] for result in all_results]
url_keyword_list = [{'link': result['link'], 'keyword': result['keyword']} for result in all_results]



crawler_threads = []
error_links = []

before_starting = datetime.now()

links_to_crawl=[]


for item in url_keyword_list:
    link = item['link']
    keyword = item['keyword']
    print(f"Scraping: {link} for keyword: {keyword}")
    scrape_and_save(link, keyword, conf, crawled_csv)
time.sleep(10)
run(conf['dir'] + "output.json")

The Crawler has the following configuraions:
{'dir': '/content/drive/My Drive/KAIST/Spider1/', 'imgsDir': '/content/drive/My Drive/KAIST/Spider1/imgs/', 'keywords': ['Elon Musk', 'Tour Eiffel'], 'labels': ['technology', 'business', 'politics', 'history'], 'blacklist': ['youtube.com', 'facebook.com', 'twitter.com', 'x.com', 'instagram.com', 'linkedin.com', 'tiktok.com'], 'mode': 'summarize', 'image_weight': 0.3}
Searching for keyword: Elon Musk
Searching for keyword: Tour Eiffel
Failed to scrape the page: No links found — possibly blocked or unexpected page structure.
Retries left: 99
Scraping: https://en.wikipedia.org/wiki/Elon_Musk for keyword: Elon Musk
Done scraping https://en.wikipedia.org/wiki/Elon_Musk
Scraping: https://abcnews.go.com/Politics/trump-disappointed-elon-musk-musk-strikes-back-real/story?id=122543215 for keyword: Elon Musk
Done scraping https://abcnews.go.com/Politics/trump-disappointed-elon-musk-musk-strikes-back-real/story?id=122543215
Scraping: https://www.biograp

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Image <PIL.Image.Image image mode=RGB size=473x60 at 0x7DBD3126F990> is not relevant with similarity 0.1946


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Image <PIL.Image.Image image mode=RGB size=361x55 at 0x7DBD3126F990> is not relevant with similarity 0.1844
Image <PIL.Image.Image image mode=RGB size=361x55 at 0x7DBD9A284250> is not relevant with similarity 0.1844
Image <PIL.Image.Image image mode=RGB size=436x450 at 0x7DBD9A28AF90> is not relevant with similarity 0.2907
Image <PIL.Image.Image image mode=RGB size=550x367 at 0x7DBD9A8F8AD0> is not relevant with similarity 0.2755
Image <PIL.Image.Image image mode=RGB size=180x300 at 0x7DBD9A21A910> is relevant with similarity 0.3018
Image <PIL.Image.Image image mode=RGB size=296x450 at 0x7DBD9A218310> is relevant with similarity 0.3156
Image <PIL.Image.Image image mode=RGB size=342x450 at 0x7DBD9AA61A10> is relevant with similarity 0.3023
Image <PIL.Image.Image image mode=RGB size=300x450 at 0x7DBD9A287D50> is relevant with similarity 0.3038
Image <PIL.Image.Image image mode=RGB size=367x550 at 0x7DBD9A219B10> is not relevant with similarity 0.2986
Image <PIL.Image.Image image mode=RGB

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Image <PIL.Image.Image image mode=RGB size=600x113 at 0x7DBD9A8F8AD0> is not relevant with similarity 0.1980
Image <PIL.Image.Image image mode=RGB size=2000x1053 at 0x7DBD9A241490> is not relevant with similarity 0.2732
Image <PIL.Image.Image image mode=RGB size=150x150 at 0x7DBD9A219A90> is not relevant with similarity 0.2898
Image <PIL.Image.Image image mode=RGB size=667x1000 at 0x7DBD8EB4FC90> is relevant with similarity 0.3105
Image <PIL.Image.Image image mode=RGB size=1500x1001 at 0x7DBD9A285110> is not relevant with similarity 0.2694
Image <PIL.Image.Image image mode=RGB size=1500x1000 at 0x7DBD9A21AE50> is not relevant with similarity 0.2267
Image <PIL.Image.Image image mode=RGB size=1500x1000 at 0x7DBD9A21A9D0> is not relevant with similarity 0.2445
Image <PIL.Image.Image image mode=RGB size=1500x1001 at 0x7DBD311FA3D0> is not relevant with similarity 0.1902
Image <PIL.Image.Image image mode=RGB size=667x1000 at 0x7DBD9A2875D0> is not relevant with similarity 0.1761
Image <PIL.

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Image <PIL.Image.Image image mode=RGB size=900x120 at 0x7DBD3120AC90> is not relevant with similarity 0.1828
Image <PIL.Image.Image image mode=RGB size=900x120 at 0x7DBD9A1F5390> is not relevant with similarity 0.1828
Image <PIL.Image.Image image mode=RGB size=1024x1024 at 0x7DBD9A243B90> is not relevant with similarity 0.2299
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD8E125C10> is relevant with similarity 0.3059
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD9A288F10> is relevant with similarity 0.3034
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD8E191090> is not relevant with similarity 0.2442
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD9A287B10> is relevant with similarity 0.3049
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD9AA61A10> is not relevant with similarity 0.2909
Image <PIL.Image.Image image mode=RGB size=768x1024 at 0x7DBD9A20B3D0> is relevant with similarity 0.3004
Image <PIL.Image.Image imag